<a href="https://colab.research.google.com/github/morioka/LlamaIndex_trial/blob/main/llmaindex_question_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LlamaIndexを用いて、クイズ生成を考える

- インデックスを作成する
- BeautifulSoupをつかって記事本文だけをインデックス
- 解答候補を抽出する
- 関連情報を得る
- 質問生成：四択問題
- 検証: 質問＋各選択肢が成立するか
- 導入文の生成
- 導入文と質問・解答をペアにしてすわりがよいかを評価
- クイズの難しさの評価
- クイズとしての面白さの評価







- [ChatGPTで独自データを利用できるLlamaIndexはどんな仕組みで動いているのか？調べてみました | DevelopersIO](https://dev.classmethod.jp/articles/llamaindex-overview/)

## 対象の記事

In [1]:
# この例では、この記事を使います

article_urls=['https://nordot.app/1014472568104288256']  # 細野勇策が通算16アンダー首位　東建男子ゴルフ第2日 ｜ 共同通信 (2023/03/31)

In [ ]:
# nordotから一覧

from urllib import request  # urllib.requestモジュールをインポート
import json

url = 'https://nordot.app/-/units/39166665832988672/list?offset=10&limit=10'
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'   # User-Agentを設定しないと応答を返さない
headers = {'User-Agent': user_agent}

req = request.Request(url, headers=headers)
with request.urlopen(req) as response:
   content = json.loads(response.read().decode('utf8'))


In [ ]:
[
    (p["title"], p["published_at"], f'https://nordot.app/{p["id"]}') for p in content['posts']
]


[('米2月物価、5％上昇',
  '2023-03-31T13:07:43+00:00',
  'https://nordot.app/1014522080913571840'),
 ('NY円、132円台後半',
  '2023-03-31T13:06:07+00:00',
  'https://nordot.app/1014521617494933504'),
 ('トランプ氏、4日にも出頭',
  '2023-03-31T13:05:53+00:00',
  'https://nordot.app/1014495251956924416'),
 ('不正閲覧、改善命令を勧告',
  '2023-03-31T12:58:56+00:00',
  'https://nordot.app/1014499439336554496'),
 ('西2―3オ（31日）',
  '2023-03-31T12:56:03+00:00',
  'https://nordot.app/1014513667462299648'),
 ('WBC優勝の熱気つなげる',
  '2023-03-31T12:55:41+00:00',
  'https://nordot.app/1014518304881295360'),
 ('前大統領起訴「ノーコメント」とバイデン氏',
  '2023-03-31T12:54:48+00:00',
  'https://nordot.app/1014518711919214592'),
 ('厚労省、「エホバ」と面会し要請',
  '2023-03-31T12:54:37+00:00',
  'https://nordot.app/1014499176452636672'),
 ('神6―3D（31日）',
  '2023-03-31T12:54:24+00:00',
  'https://nordot.app/1014518178649620480'),
 ('米国との戦略核の均衡保つとロ外交政策概念',
  '2023-03-31T12:50:41+00:00',
  'https://nordot.app/1014517638885523456')]

## デバッグログ設定

In [2]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

## インデックスを作成する


In [3]:
!pip install -qU llama-index wikipedia langchain html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 KB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.0/466.0 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 5.7 MB/s eta 0:00:00


In [4]:
!pip install -qU gpt-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 KB 9.6 MB/s eta 0:00:00


In [5]:
import os
os.environ["OPENAI_API_KEY"] = 'YOUR_OPENAI_API_KEY'

In [9]:
from llama_index import GPTSimpleVectorIndex, SimpleDirectoryReader
#documents = SimpleDirectoryReader('data').load_data()
#index = GPTSimpleVectorIndex.from_documents(documents)

In [10]:
from llama_index import GPTSimpleVectorIndex, SimpleWebPageReader

documents = SimpleWebPageReader().load_data(article_urls)
index = GPTSimpleVectorIndex.from_documents(documents)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): nordot.app:443
DEBUG:urllib3.connectionpool:https://nordot.app:443 "GET /1014472568104288256 HTTP/1.1" 200 None
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: <!doctype html>

  <html lang="ja-JP">
  <head ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk:      <div class="header__sns">
          <div c...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: target="_blank">
      <span class="shareButton...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 1 0 0 1 .707.293l7 7a1 1 0 0 1 0 1.414l-4.586 4...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk:                        
                       ...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk:    <path class="cls-1" d="M170.46,63.44A22.18,2...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: function(){
        nor.countRevContentLoadMore...
DEBUG:openai:message='Request to OpenAI API' method=post path=https:

In [11]:
index.query("<question_text>?")

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["<question_text>?"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=105 request_id=bc85a32c2eae4e38d3399e466ecf99ed response_code=200
DEBUG:llama_index.indices.utils:> Top 1 nodes:
> [Node adedafd4-efd5-4a3b-836b-b69b4ee3d467] [Similarity score:             0.734375]    <path class="cls-1" d="M170.46,63.44A22.18,22.18,0,0,1,162,61.77a20.89,20.89,0,0,1-7.25-5,24.6...
DEBUG:llama_index.indices.query.base:> Searching in chunk:    <path class="cls-1" d="M170.46,63.44A22.18,2...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.

Response(response='\nThe answer to the question is not provided in the context information.', source_nodes=[NodeWithScore(node=Node(text='   <path class="cls-1" d="M170.46,63.44A22.18,22.18,0,0,1,162,61.77a20.89,20.89,0,0,1-7.25-5,24.69,24.69,0,0,1-5.06-8.08,29.6,29.6,0,0,1-1.91-11v-.18a29.6,29.6,0,0,1,1.91-11,25.27,25.27,0,0,1,5-8.08,20.47,20.47,0,0,1,7.2-5,22.3,22.3,0,0,1,8.55-1.67A17.68,17.68,0,0,1,179.79,14a24.73,24.73,0,0,1,6.18,5.2v-18A1.25,1.25,0,0,1,187.21,0h11.64a1.25,1.25,0,0,1,1.24,1.24v60a1.25,1.25,0,0,1-1.24,1.24H187.21A1.25,1.25,0,0,1,186,61.27V55.36a22.2,22.2,0,0,1-6.27,5.8A17.24,17.24,0,0,1,170.46,63.44Zm3.62-12a11.36,11.36,0,0,0,4.69-1,11.22,11.22,0,0,0,3.81-2.78,14,14,0,0,0,2.6-4.37,15.64,15.64,0,0,0,1-5.62v-.18a15.6,15.6,0,0,0-1-5.62,13.89,13.89,0,0,0-2.6-4.37,11.12,11.12,0,0,0-3.81-2.79,11.82,11.82,0,0,0-9.38,0,11.56,11.56,0,0,0-3.85,2.79,13.47,13.47,0,0,0-2.65,4.32,15.53,15.53,0,0,0-1,5.67v.18a15.64,15.64,0,0,0,1,5.62,13.39,13.39,0,0,0,2.65,4.37,11.67,11.67,0,0,0,3

In [12]:
index.query("次の問いに日本語で答えてください。単語で出力してください。\n\n誰が首位?").response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u6b21\\u306e\\u554f\\u3044\\u306b\\u65e5\\u672c\\u8a9e\\u3067\\u7b54\\u3048\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002\\u5358\\u8a9e\\u3067\\u51fa\\u529b\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002  \\u8ab0\\u304c\\u9996\\u4f4d?"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=21 request_id=9bbabc0a92094e16970ddcf47759ae97 response_code=200
DEBUG:llama_index.indices.utils:> Top 1 nodes:
> [Node 48bfa574-9778-4ae6-89c9-c5593565490f] [Similarity score:             0.776248] <!doctype html>

  <html lang="ja-JP">
  <head prefix="og: http://ogp.me/ns

'細野勇策が共同通信の首位にいる'

In [13]:
index.query("要約してください").response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u8981\\u7d04\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=17 request_id=2e7e2cca64bdf6e7028b73fded7f51b8 response_code=200
DEBUG:llama_index.indices.utils:> Top 1 nodes:
> [Node 48bfa574-9778-4ae6-89c9-c5593565490f] [Similarity score:             0.771705] <!doctype html>

  <html lang="ja-JP">
  <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns...
DEBUG:llama_index.indices.query.base:> Searching in chunk: <!doctype html>

  <html lang="ja-JP">
  <head ...
DEBUG:openai:message='Request to OpenAI AP

'このウェブページは、共同通信の記事で、20歳の細野勇策が東建ホームメイト・カップ第2日で16アンダー首位を獲得したことを報じています。'

## BeautifulSoupをつかって記事本文だけをインデックス

こちらの方がhtmlタグなどのノイズが少ない。きれいな情報を取得でき、それを使った結果もよさそう。

In [14]:
!pip install -qU bs4

  Preparing metadata (setup.py) ... done


In [15]:
from urllib import request
from bs4 import BeautifulSoup

url = 'https://nordot.app/1014495251956924416'

url = article_urls[0]
response = request.urlopen(url)
soup = BeautifulSoup(response)
response.close()

In [16]:
text = soup.find('div', class_='main__articleBody').text.strip().replace('\u3000', ' ')
text

'第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）20歳でツアー未勝利の細野勇策が3位から11バーディー、1ボギーの61をマークし、通算16アンダー、126で首位に立った。61は同コース開催での大会最少スコアを1打更新。4打差で前日首位の今平周吾が続いた。\n 通算10アンダーの3位に64で回った田中裕基とマイケル・ヘンドリー（ニュージーランド）がつけ、さらに1打差で石川遼らが並んだ。金谷拓実は首位から71と伸ばせず7アンダーの12位に後退。昨年優勝の香妻陣一朗は3アンダーの50位。2アンダーまでの79人が決勝ラウンドに進んだ。'

In [17]:
from gpt_index import StringIterableReader, GPTTreeIndex

documents = StringIterableReader().load_data([text])
index = GPTSimpleVectorIndex.from_documents(documents)

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u7b2c2\\u65e5\\u300112\\u756a\\u3067\\u30a2\\u30d7\\u30ed\\u30fc\\u30c1\\u30b7\\u30e7\\u30c3\\u30c8\\u3092\\u653e\\u3064\\u7d30\\u91ce\\u52c7\\u7b56\\u3002\\u901a\\u7b9716\\u30a2\\u30f3\\u30c0\\u30fc\\u3067\\u9996\\u4f4d\\uff1d\\u6771\\u5efa\\u591a\\u5ea6CC\\u540d\\u53e4\\u5c4b \\u6771\\u5efa\\u30db\\u30fc\\u30e0\\u30e1\\u30a4\\u30c8\\u30fb\\u30ab\\u30c3\\u30d7\\u7b2c2\\u65e5\\uff0831\\u65e5\\u30fb\\u4e09\\u91cd\\u770c\\u6771\\u5efa\\u591a\\u5ea6CC\\u540d\\u53e4\\u5c4b\\uff1d7062\\u30e4\\u30fc\\u30c9\\u3001\\u30d1\\u30fc71\\uff0920\\u6b73\\u3067\\u30c4\\u30a2\\u30fc\\u672a\\u52dd\\u5229\\u306e\\u7d30\\u91ce\\u52c7\\u7b56\\u304c3\\u4f4d\\u304b\\u308911\\u30d0\\u30fc\\u30c7\\u30a3\\u30fc\\u30011\\u30dc\\u

In [18]:
index.query("<question_text>?")

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["<question_text>?"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=115 request_id=95fd0cd6da4a5afeb6e6190722b2c746 response_code=200
DEBUG:llama_index.indices.utils:> Top 1 nodes:
> [Node c6e8f9bd-ac3f-4472-8ea9-d78844d30a2d] [Similarity score:             0.68405] 第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）20歳で...
DEBUG:llama_index.indices.query.base:> Searching in chunk: 第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.c

Response(response='\n細野勇策は、第2日の試合で通算16アンダーで首位に立った。', source_nodes=[NodeWithScore(node=Node(text='第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）20歳でツアー未勝利の細野勇策が3位から11バーディー、1ボギーの61をマークし、通算16アンダー、126で首位に立った。61は同コース開催での大会最少スコアを1打更新。4打差で前日首位の今平周吾が続いた。\n 通算10アンダーの3位に64で回った田中裕基とマイケル・ヘンドリー（ニュージーランド）がつけ、さらに1打差で石川遼らが並んだ。金谷拓実は首位から71と伸ばせず7アンダーの12位に後退。昨年優勝の香妻陣一朗は3アンダーの50位。2アンダーまでの79人が決勝ラウンドに進んだ。', doc_id='45def042-1254-47e9-94cd-d30b3c0a667a', embedding=None, doc_hash='e28622816e464dca564852727a35e9fd913f41db896c4d57d83e345f1db0d079', extra_info=None, node_info={'start': 0, 'end': 334}, relationships={<DocumentRelationship.SOURCE: '1'>: '234bfcaa-e5c7-4607-bb6b-36737d3482eb'}), score=0.6840500336242247)], extra_info=None)

In [19]:
index.query("次の問いに日本語で答えてください。単語で出力してください。\n\n誰が首位ですか?").response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u6b21\\u306e\\u554f\\u3044\\u306b\\u65e5\\u672c\\u8a9e\\u3067\\u7b54\\u3048\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002\\u5358\\u8a9e\\u3067\\u51fa\\u529b\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002  \\u8ab0\\u304c\\u9996\\u4f4d\\u3067\\u3059\\u304b?"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=19 request_id=06eec9b6d59a5e7b04e08ce1177a983c response_code=200
DEBUG:llama_index.indices.utils:> Top 1 nodes:
> [Node c6e8f9bd-ac3f-4472-8ea9-d78844d30a2d] [Similarity score:             0.802069] 第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 東建ホームメイ

'細野勇策'

In [20]:
index.query("要約してください").response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u8981\\u7d04\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=58 request_id=c14bd534c872b498897ca1aff32bd3ff response_code=200
DEBUG:llama_index.indices.utils:> Top 1 nodes:
> [Node c6e8f9bd-ac3f-4472-8ea9-d78844d30a2d] [Similarity score:             0.767034] 第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）20歳で...
DEBUG:llama_index.indices.query.base:> Searching in chunk: 第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 ...
DEBUG:openai:message='Request to OpenAI AP

'細野勇策が3位から11バーディー、1ボギーの61で通算16アンダー、126で首位に立ち、大会最少スコアを1打更新した。田中裕基とマイケル・ヘンドリー（ニュージーランド）が通算10アンダーの3位、石川遼らが1打差で4位に入った。金谷拓実は71で7アンダーの12位、香妻陣一朗は3アンダーの50位であった。2アンダーまでの79人が決勝ラウンドに進んだ。'

## 一度保存

In [21]:
index.save_to_disk('index.json')

In [22]:
import json

with open('index.json') as f:
  index_json = json.loads(f.read())

index_json

{'index_struct': {'__type__': 'simple_dict',
  '__data__': {'index_id': '1638ce24-3d17-48b6-8adf-95c08260c259',
   'summary': None,
   'nodes_dict': {'c6e8f9bd-ac3f-4472-8ea9-d78844d30a2d': '45def042-1254-47e9-94cd-d30b3c0a667a'},
   'doc_id_dict': {'234bfcaa-e5c7-4607-bb6b-36737d3482eb': ['c6e8f9bd-ac3f-4472-8ea9-d78844d30a2d']},
   'embeddings_dict': {'c6e8f9bd-ac3f-4472-8ea9-d78844d30a2d': [-0.034551165997982025,
     0.0024996823631227016,
     0.01928921602666378,
     0.012512299232184887,
     -0.0021733350586146116,
     0.016289597377181053,
     -0.01777551881968975,
     -0.007825394161045551,
     -0.016997840255498886,
     -0.02885744534432888,
     0.0061276936903595924,
     0.01871984452009201,
     -0.019983572885394096,
     -0.01076252106577158,
     -0.002218468114733696,
     -0.0032773613929748535,
     0.005801346153020859,
     -0.002793047809973359,
     0.010637537576258183,
     0.006297810934484005,
     -0.009137728251516819,
     0.0259411484003067,
     

## 解答候補を抽出する

In [23]:
!pip install -qU spacy ja_ginza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 78.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.4.4 which is incompatible.


In [24]:
import spacy

nlp=spacy.load('ja_ginza')

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:jaxlib.mlir._mlir_libs:Initializing MLIR with module: _site_initialize_0
DEBUG:jaxlib.mlir._mlir_libs:Registering dialects from initializer <module 'jaxlib.mlir._mlir_libs._site_initialize_0' from '/usr/local/lib/python3.9/dist-packages/jaxlib/mlir/_mlir_libs/_site_initialize_0.so'>
DEBUG:jax._src.path:etils.epath found. Using etils.epath for file I/O.


In [25]:
summary_text = index.query("要約してください").response.strip()  # 本来は、記事そのままでよいだろう

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u8981\\u7d04\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=26 request_id=26914e8e20af3fe32013f1ad848b5fd8 response_code=200
DEBUG:llama_index.indices.utils:> Top 1 nodes:
> [Node c6e8f9bd-ac3f-4472-8ea9-d78844d30a2d] [Similarity score:             0.76701] 第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）20歳で...
DEBUG:llama_index.indices.query.base:> Searching in chunk: 第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 ...
DEBUG:openai:message='Request to OpenAI API

In [26]:
doc = nlp(summary_text)
doc

細野勇策が3位から11バーディー、1ボギーの61で通算16アンダー、126で首位に立ち、大会最少スコアを1打更新した。田中裕基とマイケル・ヘンドリーが3位につけ、石川遼らが1打差で並んだ。金谷拓実は7アンダーの12位に後退し、香妻陣一朗は3アンダーの50位であった。79人が2アンダーまでで決勝ラウンドに進んだ。

In [27]:
[ (e.text, e.label_) for e in doc.ents ]

[('細野勇策', 'Person'),
 ('3位', 'Rank'),
 ('11バーディー', 'N_Organization'),
 ('1ボギー', 'Period_Day'),
 ('61', 'Numex_Other'),
 ('16アンダー', 'N_Product'),
 ('126', 'Numex_Other'),
 ('首位', 'Rank'),
 ('1打', 'N_Event'),
 ('田中裕基', 'Person'),
 ('マイケル・ヘンドリー', 'Person'),
 ('3位', 'Rank'),
 ('石川遼', 'Person'),
 ('1打', 'N_Event'),
 ('金谷拓実', 'Person'),
 ('7アンダー', 'N_Product'),
 ('12位', 'Rank'),
 ('香妻', 'Person'),
 ('3アンダー', 'N_Product'),
 ('50位', 'Rank'),
 ('79人', 'N_Person'),
 ('2アンダー', 'N_Product')]

## 関連情報を得る

ひとまず Wikipedia 日本語版の記事サマリーを用いる。ある解答候補について関連情報を得たら、そちらに基づいて質問を生成してもよい。

- [PythonでWikipediaの情報を取得する | 分析ノート](https://analytics-note.xyz/programming/python-wikipedia/)
- [wikipedia · PyPI](https://pypi.org/project/wikipedia/)

In [28]:
import wikipedia
wikipedia.set_lang("ja")

In [29]:
for e in doc.ents:
  if e.label_ in ['Person', 'Company', 'Country']:
    print( e.text,  wikipedia.search(e.text, results=3))

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): ja.wikipedia.org:80
DEBUG:urllib3.connectionpool:http://ja.wikipedia.org:80 "GET /w/api.php?list=search&srprop=&srlimit=3&limit=3&srsearch=%E7%B4%B0%E9%87%8E%E5%8B%87%E7%AD%96&format=json&action=query HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ja.wikipedia.org:443
DEBUG:urllib3.connectionpool:https://ja.wikipedia.org:443 "GET /w/api.php?action=query&format=json&limit=3&list=search&srlimit=3&srprop=&srsearch=%E7%B4%B0%E9%87%8E%E5%8B%87%E7%AD%96 HTTP/1.1" 200 424
細野勇策 ['さすがの猿飛', '日本の写真家一覧', '岐阜県立岐阜高等学校']
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): ja.wikipedia.org:80
DEBUG:urllib3.connectionpool:http://ja.wikipedia.org:80 "GET /w/api.php?list=search&srprop=&srlimit=3&limit=3&srsearch=%E7%94%B0%E4%B8%AD%E8%A3%95%E5%9F%BA&format=json&action=query HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ja.wikipedia.org:443
DEBUG:urllib3.connectionpoo

「さすがの猿飛」は細野不二彦。おそらく間違い。

In [30]:
rel_urls = []
for e in doc.ents:
  if e.label_ in ['Person', 'Company', 'Country']:
    wp_ents = wikipedia.search(e.text, results=3)
    for wp in wp_ents:
      wp_page = wikipedia.page(wp)
      if e.text in wp_page.summary: # wp_page.contentでもよいが分量を懸念
        rel_urls.append(wp_page.summary)

rel_urls

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): ja.wikipedia.org:80
DEBUG:urllib3.connectionpool:http://ja.wikipedia.org:80 "GET /w/api.php?list=search&srprop=&srlimit=1&limit=1&srsearch=%E3%81%95%E3%81%99%E3%81%8C%E3%81%AE%E7%8C%BF%E9%A3%9B&srinfo=suggestion&format=json&action=query HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ja.wikipedia.org:443
DEBUG:urllib3.connectionpool:https://ja.wikipedia.org:443 "GET /w/api.php?action=query&format=json&limit=1&list=search&srinfo=suggestion&srlimit=1&srprop=&srsearch=%E3%81%95%E3%81%99%E3%81%8C%E3%81%AE%E7%8C%BF%E9%A3%9B HTTP/1.1" 200 215
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): ja.wikipedia.org:80
DEBUG:urllib3.connectionpool:http://ja.wikipedia.org:80 "GET /w/api.php?prop=info%7Cpageprops&inprop=url&ppprop=disambiguation&redirects=&titles=%E3%81%95%E3%81%99%E3%81%8C%E3%81%AE%E7%8C%BF%E9%A3%9B&format=json&action=query HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting 

['『石川遼スペシャル RESPECT 〜ゴルフを愛する人々へ〜』（いしかわりょうスペシャル リスペクト ゴルフをあいするひとびとへ）は、2010年4月4日から2012年3月31日まで、テレビ東京系列で放送されていたゴルフトーク番組。石川遼の冠番組。',
 '香妻 琴乃（こうづま ことの、1992年4月17日 - ）は、日本の女子プロゴルファー。鹿児島県鹿屋市出身。サマンサタバサ所属。',
 '香妻 陣一郎（こうづま じんいちろう、1994年7月7日 - ）は、日本のプロゴルファー。鹿児島県鹿屋市出身、身長165cm、日章学園高校卒業、国際スポーツ振興協会所属。妻はモデルの武井玲奈。姉は女子プロゴルファーの香妻琴乃。']

In [31]:
if False:
  documents_rel = SimpleWebPageReader().load_data(rel_urls)
  index_rel = GPTSimpleVectorIndex.from_documents(documents_rel)

In [32]:
from gpt_index import StringIterableReader, GPTTreeIndex

documents_rel = StringIterableReader().load_data(rel_urls)
index_rel = GPTSimpleVectorIndex.from_documents(documents_rel)

DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 『石川遼スペシャル RESPECT 〜ゴルフを愛する人々へ〜』（いしかわりょうスペシャル リス...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 香妻 琴乃（こうづま ことの、1992年4月17日 - ）は、日本の女子プロゴルファー。鹿児島...
DEBUG:llama_index.node_parser.node_utils:> Adding chunk: 香妻 陣一郎（こうづま じんいちろう、1994年7月7日 - ）は、日本のプロゴルファー。鹿児...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u300e\\u77f3\\u5ddd\\u907c\\u30b9\\u30da\\u30b7\\u30e3\\u30eb RESPECT \\u301c\\u30b4\\u30eb\\u30d5\\u3092\\u611b\\u3059\\u308b\\u4eba\\u3005\\u3078\\u301c\\u300f\\uff08\\u3044\\u3057\\u304b\\u308f\\u308a\\u3087\\u3046\\u30b9\\u30da\\u30b7\\u30e3\\u30eb \\u30ea\\u30b9\\u30da\\u30af\\u30c8 \\u30b4\\u30eb\\u30d5\\u3092\\u3042\\u3044\\u3059\\u308b\\u3072\\u3068\\u3073\\u3068\\u3078\\uff09\\u306f\\u30012010\\u5e744\\u67084\\u65e5\\u304b\\u30892012\\u5e743\\u670831\\u65e5\\u307e\\u3067\\u3001\\u30

## 質問生成 四択問題

In [33]:
index.query("あなたは有能なクイズ作家です。細野勇作が正解になる四択クイズを作ってください。").response.strip()  # 実は名前を間違い

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u7d30\\u91ce\\u52c7\\u4f5c\\u304c\\u6b63\\u89e3\\u306b\\u306a\\u308b\\u56db\\u629e\\u30af\\u30a4\\u30ba\\u3092\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=21 request_id=d91f6013908f814f5e2dca6cf3a99b17 response_code=200
DEBUG:llama_index.indices.utils:> Top 1 nodes:
> [Node c6e8f9bd-ac3f-4472-8ea9-d78844d30a2d] [Similarity score:             0.799846] 第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位

'Q. 第2日の大会で首位に立ったのは誰でしたか？\nA. 1. 今平周吾  2. 細野勇策  3. 田中裕基  4. マイケル・ヘンドリー'

In [34]:
index.query("あなたは有能なクイズ作家です。細野勇策が正解になる四択クイズを作ってください。").response.strip() 

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u7d30\\u91ce\\u52c7\\u7b56\\u304c\\u6b63\\u89e3\\u306b\\u306a\\u308b\\u56db\\u629e\\u30af\\u30a4\\u30ba\\u3092\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=32 request_id=80f0d8ee7c3d8079999fba18e9a9b723 response_code=200
DEBUG:llama_index.indices.utils:> Top 1 nodes:
> [Node c6e8f9bd-ac3f-4472-8ea9-d78844d30a2d] [Similarity score:             0.805195] 第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位

'Q. 細野勇策は、東建多度CC名古屋で開催された東建ホームメイト・カップ第2日で何をマークしましたか？\nA. \nA. 61\nB. 64\nC. 71\nD. 126'

In [35]:
index.query("あなたは有能なクイズ作家です。細野勇策が正解になる四択クイズを作ってください。3つ作ってください").response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u7d30\\u91ce\\u52c7\\u7b56\\u304c\\u6b63\\u89e3\\u306b\\u306a\\u308b\\u56db\\u629e\\u30af\\u30a4\\u30ba\\u3092\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u30023\\u3064\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=23 request_id=673d6ca8eea1ea644f74e016e233d19a response_code=200
DEBUG:llama_index.indices.utils:> Top 1 nodes:
> [Node c6e8f9bd-ac3f-4472-8ea9-d78844d30a2d] [Similarity score:  

'1. 細野勇策は、東建多度CC名古屋で開催された東建ホームメイト・カップ第2日で何スコアをマークしましたか？\nA. 59\nB. 61\nC. 63\nD. 65\n\n2. 細野勇策の通算スコアは何アンダーでしたか？\nA. 14\nB. 16\nC. 18\nD. 20\n\n3. 細野勇策が首位に立った東建ホームメイト・カップ第2日で、金谷拓実は何スコアをマークしましたか？\nA. 69\nB. 71\nC. 73\nD. 75'

生成内容の指定が不十分

## 検証: 質問＋各選択肢が成立するか

正例負例それぞれで成立不成立を問い直す

In [36]:
query = """あなたは有能なクイズ作家です。次の質問と回答のペアは適切ですか。YES/NOで答えてください。

質問: 細野勇策が通算16アンダー首位となったのはどこで行われたゴルフ大会でしたか？
回答: 東建ホームメイト・カップ
"""

In [37]:
index.query(query).response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u6b21\\u306e\\u8cea\\u554f\\u3068\\u56de\\u7b54\\u306e\\u30da\\u30a2\\u306f\\u9069\\u5207\\u3067\\u3059\\u304b\\u3002YES/NO\\u3067\\u7b54\\u3048\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002  \\u8cea\\u554f: \\u7d30\\u91ce\\u52c7\\u7b56\\u304c\\u901a\\u7b9716\\u30a2\\u30f3\\u30c0\\u30fc\\u9996\\u4f4d\\u3068\\u306a\\u3063\\u305f\\u306e\\u306f\\u3069\\u3053\\u3067\\u884c\\u308f\\u308c\\u305f\\u30b4\\u30eb\\u30d5\\u5927\\u4f1a\\u3067\\u3057\\u305f\\u304b\\uff1f \\u56de\\u7b54: \\u6771\\u5efa\\u30db\\u30fc\\u30e0\\u30e1\\u30a4\\u30c8\\u30fb\\u30ab\\u30c3\\u30d7 "], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/e

'YES'

In [38]:
query_template = """あなたは有能なクイズ作家です。次の質問と回答のペアは適切ですか。YES/NOで答えてください。


質問: {question}
回答: {answer}
"""

In [39]:
question = "第2日の大会で首位に立ったのは誰でしたか？"
answer = "今平周吾"
#answer = "細野勇策"
#answer = "田中裕基"
#answer = "マイケル・ヘンドリー"

query = query_template.format(question=question, answer=answer)
index.query(query).response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u6b21\\u306e\\u8cea\\u554f\\u3068\\u56de\\u7b54\\u306e\\u30da\\u30a2\\u306f\\u9069\\u5207\\u3067\\u3059\\u304b\\u3002YES/NO\\u3067\\u7b54\\u3048\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002   \\u8cea\\u554f: \\u7b2c2\\u65e5\\u306e\\u5927\\u4f1a\\u3067\\u9996\\u4f4d\\u306b\\u7acb\\u3063\\u305f\\u306e\\u306f\\u8ab0\\u3067\\u3057\\u305f\\u304b\\uff1f \\u56de\\u7b54: \\u4eca\\u5e73\\u5468\\u543e "], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=29 reques

'No. The correct answer is 細野勇策.'

In [40]:
question = "第2日の大会で首位に立ったのは誰でしたか？"
answer = "今平周吾"
answer = "細野勇策"
#answer = "田中裕基"
#answer = "マイケル・ヘンドリー"

query = query_template.format(question=question, answer=answer)
index.query(query).response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u6b21\\u306e\\u8cea\\u554f\\u3068\\u56de\\u7b54\\u306e\\u30da\\u30a2\\u306f\\u9069\\u5207\\u3067\\u3059\\u304b\\u3002YES/NO\\u3067\\u7b54\\u3048\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002   \\u8cea\\u554f: \\u7b2c2\\u65e5\\u306e\\u5927\\u4f1a\\u3067\\u9996\\u4f4d\\u306b\\u7acb\\u3063\\u305f\\u306e\\u306f\\u8ab0\\u3067\\u3057\\u305f\\u304b\\uff1f \\u56de\\u7b54: \\u7d30\\u91ce\\u52c7\\u7b56 "], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=19 reques

'YES'

In [41]:
question = "第2日の大会で首位に立ったのは誰でしたか？"
answer = "今平周吾"
answer = "細野勇策"
answer = "田中裕基"
#answer = "マイケル・ヘンドリー"

query = query_template.format(question=question, answer=answer)
index.query(query).response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u6b21\\u306e\\u8cea\\u554f\\u3068\\u56de\\u7b54\\u306e\\u30da\\u30a2\\u306f\\u9069\\u5207\\u3067\\u3059\\u304b\\u3002YES/NO\\u3067\\u7b54\\u3048\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002   \\u8cea\\u554f: \\u7b2c2\\u65e5\\u306e\\u5927\\u4f1a\\u3067\\u9996\\u4f4d\\u306b\\u7acb\\u3063\\u305f\\u306e\\u306f\\u8ab0\\u3067\\u3057\\u305f\\u304b\\uff1f \\u56de\\u7b54: \\u7530\\u4e2d\\u88d5\\u57fa "], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=43 reques

'NO'

In [42]:
question = "第2日の大会で首位に立ったのは誰でしたか？"
answer = "今平周吾"
answer = "細野勇策"
answer = "田中裕基"
answer = "マイケル・ヘンドリー"

query = query_template.format(question=question, answer=answer)
index.query(query).response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u6b21\\u306e\\u8cea\\u554f\\u3068\\u56de\\u7b54\\u306e\\u30da\\u30a2\\u306f\\u9069\\u5207\\u3067\\u3059\\u304b\\u3002YES/NO\\u3067\\u7b54\\u3048\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002   \\u8cea\\u554f: \\u7b2c2\\u65e5\\u306e\\u5927\\u4f1a\\u3067\\u9996\\u4f4d\\u306b\\u7acb\\u3063\\u305f\\u306e\\u306f\\u8ab0\\u3067\\u3057\\u305f\\u304b\\uff1f \\u56de\\u7b54: \\u30de\\u30a4\\u30b1\\u30eb\\u30fb\\u30d8\\u30f3\\u30c9\\u30ea\\u30fc "], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-

'No. 第2日の大会で首位に立ったのは細野勇策でした。'

## 導入文の生成

In [43]:
query_template = """あなたは有能なクイズ作家です。次の質問と回答のペアに対して、クイズとして適切な導入文を付与してください。導入文には、質問の内容を含みません。導入文は疑問文ではありません。導入文には、解答の内容を含みません。 5つ、作成してください。

質問: {question}
回答: {answer}
"""

In [44]:
question = "細野勇策が通算16アンダー首位となったのはどこで行われたゴルフ大会でしたか？"
answer = "東建ホームメイト・カップ"

In [45]:
query = query_template.format(question=question, answer=answer)

query

'あなたは有能なクイズ作家です。次の質問と回答のペアに対して、クイズとして適切な導入文を付与してください。導入文には、質問の内容を含みません。導入文は疑問文ではありません。導入文には、解答の内容を含みません。 5つ、作成してください。\n\n質問: 細野勇策が通算16アンダー首位となったのはどこで行われたゴルフ大会でしたか？\n回答: 東建ホームメイト・カップ\n'

In [46]:
index.query(query).response.strip().replace('\n','')

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u6b21\\u306e\\u8cea\\u554f\\u3068\\u56de\\u7b54\\u306e\\u30da\\u30a2\\u306b\\u5bfe\\u3057\\u3066\\u3001\\u30af\\u30a4\\u30ba\\u3068\\u3057\\u3066\\u9069\\u5207\\u306a\\u5c0e\\u5165\\u6587\\u3092\\u4ed8\\u4e0e\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002\\u5c0e\\u5165\\u6587\\u306b\\u306f\\u3001\\u8cea\\u554f\\u306e\\u5185\\u5bb9\\u3092\\u542b\\u307f\\u307e\\u305b\\u3093\\u3002\\u5c0e\\u5165\\u6587\\u306f\\u7591\\u554f\\u6587\\u3067\\u306f\\u3042\\u308a\\u307e\\u305b\\u3093\\u3002\\u5c0e\\u5165\\u6587\\u306b\\u306f\\u3001\\u89e3\\u7b54\\u306e\\u5185\\u5bb9\\u3092\\u542b\\u307f\\u307e\\u305b\\u3093\\u3002 5\\u3064\\u3001\\u4f5c\\u6210\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002  \\u8cea\\u554f: \\u7

'1. 東建ホームメイト・カップで細野勇策が通算16アンダーで首位を獲得しました。2. 細野勇策が通算16アンダーで首位を獲得したのは、東建ホームメイト・カップでした。3. 東建ホームメイト・カップで細野勇策が通算16アンダーで首位を獲得しました。4. 細野勇策が東建ホームメイト・カップで通算16アンダーで首位を獲得しました。5. 東建ホームメイト・カップで細'

In [47]:
index_rel.query(query).response.strip().replace('\n','')

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u6b21\\u306e\\u8cea\\u554f\\u3068\\u56de\\u7b54\\u306e\\u30da\\u30a2\\u306b\\u5bfe\\u3057\\u3066\\u3001\\u30af\\u30a4\\u30ba\\u3068\\u3057\\u3066\\u9069\\u5207\\u306a\\u5c0e\\u5165\\u6587\\u3092\\u4ed8\\u4e0e\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002\\u5c0e\\u5165\\u6587\\u306b\\u306f\\u3001\\u8cea\\u554f\\u306e\\u5185\\u5bb9\\u3092\\u542b\\u307f\\u307e\\u305b\\u3093\\u3002\\u5c0e\\u5165\\u6587\\u306f\\u7591\\u554f\\u6587\\u3067\\u306f\\u3042\\u308a\\u307e\\u305b\\u3093\\u3002\\u5c0e\\u5165\\u6587\\u306b\\u306f\\u3001\\u89e3\\u7b54\\u306e\\u5185\\u5bb9\\u3092\\u542b\\u307f\\u307e\\u305b\\u3093\\u3002 5\\u3064\\u3001\\u4f5c\\u6210\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002  \\u8cea\\u554f: \\u7

'1. 細野勇策がゴルフ大会で何を達成したのかご存知ですか？2. 東建ホームメイト・カップで細野勇策が何を達成したのかご存知ですか？3. 細野勇策がゴルフ大会で何を達成したと話題になっていますか？4. 細野勇策がゴルフ大会で何を達成したのか、ご存知ですか？5. 細野勇策がゴルフ大会で何を達成したのか、ご存じですか？'

## 導入文と質問・解答をペアにしてすわりがよいかを評価
## クイズの難しさの評価
## クイズとしての面白さの評価


TBD

In [48]:
query_template = """あなたは有能なクイズ作家です。次の質問と回答のペアの難しさを1から5までの5段階で評価してください。簡単であれば1を、難しければ5を出力してください。


質問: {question}
回答: {answer}
"""

In [49]:
question = "第2日の大会で首位に立ったのは誰でしたか？"
answer = "今平周吾"
answer = "細野勇策"
#answer = "田中裕基"
#answer = "マイケル・ヘンドリー"

query = query_template.format(question=question, answer=answer)
index.query(query).response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u6b21\\u306e\\u8cea\\u554f\\u3068\\u56de\\u7b54\\u306e\\u30da\\u30a2\\u306e\\u96e3\\u3057\\u3055\\u30921\\u304b\\u30895\\u307e\\u3067\\u306e5\\u6bb5\\u968e\\u3067\\u8a55\\u4fa1\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002\\u7c21\\u5358\\u3067\\u3042\\u308c\\u30701\\u3092\\u3001\\u96e3\\u3057\\u3051\\u308c\\u30705\\u3092\\u51fa\\u529b\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002   \\u8cea\\u554f: \\u7b2c2\\u65e5\\u306e\\u5927\\u4f1a\\u3067\\u9996\\u4f4d\\u306b\\u7acb\\u3063\\u305f\\u306e\\u306f\\u8ab0\\u3067\\u3057\\u305f\\u304b\\uff1f \\u56de\\u7b54: \\u7d30\\u91ce\\u52c7\\u7b56 "], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v

'1'

In [50]:
query = """あなたは有能なクイズ作家です。細野勇策を解答とする、難しさ5の四択クイズを作ってください。難しさは1から5までの5段階です。簡単であれば1を、難しければ5です。
"""
index.query(query).response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u7d30\\u91ce\\u52c7\\u7b56\\u3092\\u89e3\\u7b54\\u3068\\u3059\\u308b\\u3001\\u96e3\\u3057\\u30555\\u306e\\u56db\\u629e\\u30af\\u30a4\\u30ba\\u3092\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002\\u96e3\\u3057\\u3055\\u306f1\\u304b\\u30895\\u307e\\u3067\\u306e5\\u6bb5\\u968e\\u3067\\u3059\\u3002\\u7c21\\u5358\\u3067\\u3042\\u308c\\u30701\\u3092\\u3001\\u96e3\\u3057\\u3051\\u308c\\u30705\\u3067\\u3059\\u3002 "], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddin

'Q. 細野勇策は、東建多度CC名古屋で開催された東建ホームメイト・カップ第2日で何スコアをマークしましたか？\n\nA. 1. 68\n2. 69\n3. 61\n4. 62'

In [51]:
query = """あなたは有能なクイズ作家です。細野勇策を解答とする、難しさ1の四択クイズを作ってください。難しさは1から5までの5段階です。簡単であれば1を、難しければ5です。
"""
index.query(query).response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u7d30\\u91ce\\u52c7\\u7b56\\u3092\\u89e3\\u7b54\\u3068\\u3059\\u308b\\u3001\\u96e3\\u3057\\u30551\\u306e\\u56db\\u629e\\u30af\\u30a4\\u30ba\\u3092\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002\\u96e3\\u3057\\u3055\\u306f1\\u304b\\u30895\\u307e\\u3067\\u306e5\\u6bb5\\u968e\\u3067\\u3059\\u3002\\u7c21\\u5358\\u3067\\u3042\\u308c\\u30701\\u3092\\u3001\\u96e3\\u3057\\u3051\\u308c\\u30705\\u3067\\u3059\\u3002 "], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddin

'Q. 細野勇策は、東建多度CC名古屋で開催された東建ホームメイト・カップ第2日で何スコアをマークしましたか？\n\nA. \nA. 61\nB. 64\nC. 71\nD. 79'

In [52]:
query = """あなたは有能なクイズ作家です。細野勇策を解答とする、難しさ3の四択クイズを作ってください。難しさは1から5までの5段階です。簡単であれば1を、難しければ5です。
"""
index.query(query).response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u6709\\u80fd\\u306a\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u7d30\\u91ce\\u52c7\\u7b56\\u3092\\u89e3\\u7b54\\u3068\\u3059\\u308b\\u3001\\u96e3\\u3057\\u30553\\u306e\\u56db\\u629e\\u30af\\u30a4\\u30ba\\u3092\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002\\u96e3\\u3057\\u3055\\u306f1\\u304b\\u30895\\u307e\\u3067\\u306e5\\u6bb5\\u968e\\u3067\\u3059\\u3002\\u7c21\\u5358\\u3067\\u3042\\u308c\\u30701\\u3092\\u3001\\u96e3\\u3057\\u3051\\u308c\\u30705\\u3067\\u3059\\u3002 "], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddin

'Q. 細野勇策は、東建多度CC名古屋で開催された東建ホームメイト・カップ第2日で何スコアをマークしましたか？\n\nA. \nA. 61\nB. 64\nC. 71\nD. 79'

クイズの難しさについて、何か言えているわけではなさそう。いくつか例示が必要なのだろう。

あとは、マルチホップが必要な質問を作れるか？その確認のためにマルチホップQAができるか。どちらもステップバイステップのCoT(Chain of Thoughts)でなんとかならないか?

----


In [53]:
index.query("""あなたはプロのクイズ作家です。特に四択問題を作ることを得意としています。 以下のテーマでクイズを作ってください。

テーマ：ボーイング737MAX""").response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u30d7\\u30ed\\u306e\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u7279\\u306b\\u56db\\u629e\\u554f\\u984c\\u3092\\u4f5c\\u308b\\u3053\\u3068\\u3092\\u5f97\\u610f\\u3068\\u3057\\u3066\\u3044\\u307e\\u3059\\u3002 \\u4ee5\\u4e0b\\u306e\\u30c6\\u30fc\\u30de\\u3067\\u30af\\u30a4\\u30ba\\u3092\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002  \\u30c6\\u30fc\\u30de\\uff1a\\u30dc\\u30fc\\u30a4\\u30f3\\u30b0737MAX"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=20 request_id=0d61b91864058ba9db9f52951c9f4813 resp

'Q. 日本で初めてボーイング737MAXを導入した航空会社はどこですか？\n\nA. 日本で初めてボーイング737MAXを導入した航空会社は、日本航空（JAL）です。'

In [54]:
index.query("""あなたはプロのクイズ作家です。特に四択問題を作ることを得意としています。 以下のテーマで四択クイズを作ってください。

テーマ：細野勇策""").response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u30d7\\u30ed\\u306e\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u7279\\u306b\\u56db\\u629e\\u554f\\u984c\\u3092\\u4f5c\\u308b\\u3053\\u3068\\u3092\\u5f97\\u610f\\u3068\\u3057\\u3066\\u3044\\u307e\\u3059\\u3002 \\u4ee5\\u4e0b\\u306e\\u30c6\\u30fc\\u30de\\u3067\\u56db\\u629e\\u30af\\u30a4\\u30ba\\u3092\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002  \\u30c6\\u30fc\\u30de\\uff1a\\u7d30\\u91ce\\u52c7\\u7b56"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=132 request_id=8a754e43b4c1012490a60b44c33b4d42 re

'Q. 細野勇策は、東建多度CC名古屋で開催された東建ホームメイト・カップ第2日で何をマークしましたか？\n\nA.\nA. 61'

In [55]:
index.query("""あなたはプロのクイズ作家です。特に四択問題を作ることを得意としています。 以下のテーマで四択クイズを作ってください。

テーマ：東建""").response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u30d7\\u30ed\\u306e\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u7279\\u306b\\u56db\\u629e\\u554f\\u984c\\u3092\\u4f5c\\u308b\\u3053\\u3068\\u3092\\u5f97\\u610f\\u3068\\u3057\\u3066\\u3044\\u307e\\u3059\\u3002 \\u4ee5\\u4e0b\\u306e\\u30c6\\u30fc\\u30de\\u3067\\u56db\\u629e\\u30af\\u30a4\\u30ba\\u3092\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002  \\u30c6\\u30fc\\u30de\\uff1a\\u6771\\u5efa"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=80 request_id=d66fbf6e3ba9dc78a80bf809c2a37277 response_code=200

'Q. 東建多度CC名古屋で開催された東建ホームメイト・カップ第2日で、最少スコアを記録したのは誰でしたか？\n\nA. 細野勇策'

In [56]:
index.query("""あなたはプロのクイズ作家です。特に四択問題を作ることを得意としています。 以下のテーマで四択クイズを作ってください。3つ作ってください。

テーマ：東建""").response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u30d7\\u30ed\\u306e\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u7279\\u306b\\u56db\\u629e\\u554f\\u984c\\u3092\\u4f5c\\u308b\\u3053\\u3068\\u3092\\u5f97\\u610f\\u3068\\u3057\\u3066\\u3044\\u307e\\u3059\\u3002 \\u4ee5\\u4e0b\\u306e\\u30c6\\u30fc\\u30de\\u3067\\u56db\\u629e\\u30af\\u30a4\\u30ba\\u3092\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u30023\\u3064\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002  \\u30c6\\u30fc\\u30de\\uff1a\\u6771\\u5efa"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=

'1. 東建多度CC名古屋で開催された東建ホームメイト・カップの第2日において、最少スコアを記録したのは誰でしたか？\na. 金谷拓実 b. 細野勇策 c. 今平周吾 d. 石川遼\n\n2. 東建ホームメイト・カップの第2日において、首位に立ったのは誰でしたか？\na. 金谷拓実 b. 細野勇策 c. 今平周吾 d. 石川遼\n\n3. 東建ホームメイト・カップの第2日に'

In [57]:
index_rel.query("""あなたはプロのクイズ作家です。特に四択問題を作ることを得意としています。 以下のテーマで四択クイズを作ってください。3つ作ってください。

テーマ：東建""").response.strip()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u3042\\u306a\\u305f\\u306f\\u30d7\\u30ed\\u306e\\u30af\\u30a4\\u30ba\\u4f5c\\u5bb6\\u3067\\u3059\\u3002\\u7279\\u306b\\u56db\\u629e\\u554f\\u984c\\u3092\\u4f5c\\u308b\\u3053\\u3068\\u3092\\u5f97\\u610f\\u3068\\u3057\\u3066\\u3044\\u307e\\u3059\\u3002 \\u4ee5\\u4e0b\\u306e\\u30c6\\u30fc\\u30de\\u3067\\u56db\\u629e\\u30af\\u30a4\\u30ba\\u3092\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u30023\\u3064\\u4f5c\\u3063\\u3066\\u304f\\u3060\\u3055\\u3044\\u3002  \\u30c6\\u30fc\\u30de\\uff1a\\u6771\\u5efa"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=

'1. 東建の本社はどこにありますか？\nA. 東京都 B. 神奈川県 C. 埼玉県 D. 千葉県\n\n2. 東建の設立年はいつですか？\nA. 1945年 B. 1947年 C. 1949年 D. 1951年\n\n3. 東建の代表取締役社長は誰ですか？\nA. 小林英明 B. 小林英夫 C. 小林英雄 D. 小林英樹'

あなたはプロのクイズ作家です。特に四択問題を作ることを得意としています。
以下のテーマでクイズを作ってください。

テーマ：ボーイング737MAX



ボーイング737MAXはエンジン火災の危険性の問題で一時的に運行停止になりましたか? YES/NOで答えてください

次の質問と回答のペアは正しいですか? YES/NOで答えてください。理由もお願いします。

質問: ボーイング737MAXはどのような問題で一時的に運行停止になりましたか？
回答: MCASシステムの不具合による墜落事故

次の質問と回答のペアが正しくなるよう、新しい質問を作ってください。

質問: ボーイング737MAXはどのような問題で永久に運行停止になりましたか？
回答: MCASシステムの不具合による墜落事故

次の質問と回答のペアに対して、クイズとして適切な導入文を付与してください。導入文には、質問の内容を含みません。導入文には、解答の内容を含みません。 5つ、作成してください。